# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip uninstall scikit-learn -y

#!pip install -U scikit-learn==0.24.1


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Load the data
file_path = Path('/content/drive/MyDrive/11/Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
print(df.head())
print(df.shape)


   loan_amnt  int_rate  ...  hardship_flag debt_settlement_flag
0    10500.0    0.1719  ...              N                    N
1    25000.0    0.2000  ...              N                    N
2    20000.0    0.2000  ...              N                    N
3    10000.0    0.1640  ...              N                    N
4    22000.0    0.1474  ...              N                    N

[5 rows x 86 columns]
(68817, 86)


In [7]:
lis1=[]
for i in df.columns:
  if(df[i].dtype=='object' and i!='loan_status'):
    lis1.append(i)

print(lis1)


['home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 'debt_settlement_flag']


## Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X,columns=lis1)

# Create our target
y = df['loan_status'].to_frame()


In [9]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_amnt,68817.0,16677.594562,10277.348590,1000.00,9000.0000,15000.000,24000.0000,4.000000e+04
int_rate,68817.0,0.127718,0.048130,0.06,0.0881,0.118,0.1557,3.084000e-01
installment,68817.0,480.652863,288.062432,30.89,265.7300,404.560,648.1000,1.676230e+03
annual_inc,68817.0,88213.705210,115579.958439,40.00,50000.0000,73000.000,104000.0000,8.797500e+06
dti,68817.0,21.778153,20.199244,0.00,13.8900,19.760,26.6600,9.990000e+02
...,...,...,...,...,...,...,...,...
next_pymnt_d_May-2019,68817.0,0.616839,0.486161,0.00,0.0000,1.000,1.0000,1.000000e+00
application_type_Individual,68817.0,0.860340,0.346637,0.00,1.0000,1.000,1.0000,1.000000e+00
application_type_Joint App,68817.0,0.139660,0.346637,0.00,0.0000,0.000,0.0000,1.000000e+00
hardship_flag_N,68817.0,1.000000,0.000000,1.00,1.0000,1.000,1.0000,1.000000e+00


In [10]:
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
# YOUR CODE HERE
scalar =  StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE

scalar.fit(X_train)

StandardScaler()

In [14]:
# Scale the training and testing data
# YOUR CODE HERE
scaled_X_train = scalar.transform(X_train)
scaled_X_test = scalar.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
BRFClassifier = BalancedRandomForestClassifier(random_state =1)
BRFClassifier.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=1)

In [16]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = BRFClassifier.predict(X_test)
accuracyScore = balanced_accuracy_score(y_test,y_pred)
print(accuracyScore)

0.7885466545953005


In [17]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test,y_pred)

array([[   71,    30],
       [ 2153, 14951]])

In [18]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.62     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.62     17205



In [19]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
important_features_dict = {}
for idx, val in enumerate(BRFClassifier.feature_importances_):
    column_name = X.columns[idx]
    important_features_dict[column_name] = val

important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)
print(f'5 most important features: {important_features_list[:5]}')

5 most important features: ['total_rec_prncp', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'last_pymnt_amnt']


### Easy Ensemble Classifier

In [20]:
# Train the Classifier
# YOUR CODE HERE

EEClassifier = EasyEnsembleClassifier(random_state=1)


In [21]:
EEClassifier.fit(X_train,y_train)

EasyEnsembleClassifier(random_state=1)

In [22]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = EEClassifier.predict(X_test)
accuracyScore = balanced_accuracy_score(y_test,y_pred)
print(accuracyScore)

0.9154751595365336


In [23]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test,y_pred)

array([[   94,     7],
       [ 1706, 15398]])

In [24]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.93      0.90      0.10      0.92      0.84       101
   low_risk       1.00      0.90      0.93      0.95      0.92      0.84     17104

avg / total       0.99      0.90      0.93      0.94      0.92      0.84     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    With 91% Easy ensamble classifier has highest balanced accuracy score

2. Which model had the best recall score?

    With 87 and 90 both the models has similar accuracy scores with easy ensamble in lead

3. Which model had the best geometric mean score?

    Easy Ensemble has best geometric mean score

4. What are the top three features?

    5 most important features: ['total_rec_prncp', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'last_pymnt_amnt']